In [10]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

# Definição do modelo
class FraudDetectionModel(nn.Module):
    def __init__(self, input_dim):
        super(FraudDetectionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Função de treinamento
def train_neural_network(x_train, y_train, input_dim, num_epochs=20):
    model = FraudDetectionModel(input_dim)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(x_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        if (epoch+1) % 5 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    
    return model

# Carregar e pré-processar os dados
df = pd.read_csv('../data/transactions_preprocessed.csv')

# Excluir colunas não numéricas
X = df.drop(columns=['fraud', 'transaction_date'])
y = df['fraud']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Treinar o modelo
print("Iniciando o treinamento do modelo...")
model = train_neural_network(X_train, y_train, X_train.shape[1], num_epochs=20)

# Avaliar o modelo
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    predicted = predicted.numpy()  # Convertendo para NumPy array
    y_test = y_test.numpy()  # Convertendo para NumPy array
    print("Relatório de Classificação:")
    print(classification_report(y_test, predicted))
    print("Matriz de Confusão:")
    print(confusion_matrix(y_test, predicted))


Iniciando o treinamento do modelo...
Epoch [5/20], Loss: 0.6868
Epoch [10/20], Loss: 0.6833
Epoch [15/20], Loss: 0.6806
Epoch [20/20], Loss: 0.6781


RuntimeError: Numpy is not available